In [ ]:
%pip install scheduler
%pip install python-binance
%pip install schedule

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.1.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.1.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.1.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
import requests
import time
import os
import scheduler
import schedule
import logging
import pandas as pd
from datetime import datetime
from binance.client import Client
from binance.exceptions import BinanceAPIException, BinanceRequestException

In [ ]:
# setting
API_KEY = API_SECRET = "/"
client = Client(API_KEY, API_SECRET)


symbol = "BTCUSDT"
interval = Client.KLINE_INTERVAL_5MINUTE
csv_path = "C:/Users/thswj/Desktop/새 폴더/btc_data.csv"
log_path = "C:/Users/thswj/Desktop/새 폴더/log.txt"
retry_delay = 10  # 실패 시 재시도 간격 (10초)

run_count = 0 # 실행 횟수
max_runs = 10 # 최대 실행 횟수


In [ ]:
# 최근 저장된 타임스탬프 확인
def get_last_timestamp():
    if os.path.exists(csv_path):
        df = pd.read_csv(csv_path)
        if not df.empty:
            return pd.to_datetime(df['timestamp']).max()

    return None

# 재정 데이터 요청
def fetch_btc_candle():
    for attempt in range(5):
        try:
            candles = client.get_klines(symbol = symbol, interval = interval, limit = 100)
            return candles
        except (BinanceAPIException, BinanceRequestException) as e:
            logging.error(f"API 오류 발생, {retry_delay}초 후 재시도 ({attempt + 1}/5: {e}")
            time.sleep(retry_delay)
    logging.error("API 호출 실패 - 최대 재시도 초과")
    return None

# 데이터 수집 및 저장
def collect_data():
    candle = fetch_btc_candle()
    if not candle:
        return False

    # 데이터 구조화
    df_new = []
    for c in candle:
        timestamp = pd.to_datetime(int(c[0]), unit = 'ms')

        # 중복 수집 방지
        last_ts = get_last_timestamp()
        if last_ts is not None and timestamp <= last_ts:
            logging.info(f"이미 저장된 시점 ({timestamp}) 데이터, 건너뜀")
            continue

        df_new.append({
            "timestamp": timestamp,
            "open": float(c[1]),
            "high": float(c[2]),
            "low": float(c[3]),
            "close": float(c[4]),
            "volume": float(c[5])
            })

    if not df_new:
        logging.error("새 데이터 없음")
        return True


    df_new = pd.DataFrame(df_new)
    df_new.to_csv(csv_path, mode = 'a', index=False, header = not os.path.exists(csv_path))
    logging.info(f"{len(df_new)}개의 새 데이터 저장 완료, 마지막 시점: {df_new['timestamp'].iloc[-1]}")
    # logging.info(f"새 데이터 저장 완료: {timestamp}, close={df_new['close'].iloc[0]}")
    return True

In [ ]:
# 스케줄 작업
def job():
    global run_count
    if run_count >= max_runs:
        return
    success = collect_data()
    if success:
        run_count += 1
        logging.info(f"실행 횟수: {run_count}/{max_runs}")
        print(f"실행횟수: {run_count}/{max_runs}")
    else:
        logging.error("데이터 수집 실패")

# 스케줄 설정
schedule.every(5).minutes.do(job) # 5분마다 job 실행 (5분 캔들 기준)

Every 5 minutes do job() (last run: [never], next run: 2025-11-20 15:20:18)

In [ ]:
# 메인 루프
if __name__ == '__main__':
    # 로그 설정
    logging.basicConfig(filename = log_path, level = logging.INFO,
                    format = "%(asctime)s - %(levelname)s - %(message)s")
    print("Binance BTCUSDT 5분봉 수집 시작")
    while run_count < max_runs:
        schedule.run_pending()
        time.sleep(1)
    print('데이터 수집 종료')

    # while True:
    #     schedule.run_pending()
    #     time.sleep(1)

    #     if run_count >= MAX_RUNS:
    #         print("🛑 설정된 횟수에 도달하여 프로그램 종료.")
    #         logging.info("🛑 설정된 횟수에 도달하여 프로그램 종료.")
    #         break

Binance BTCUSDT 5분봉 수집 시작
실행횟수: 1/10
실행횟수: 2/10
실행횟수: 3/10
실행횟수: 4/10
실행횟수: 5/10
실행횟수: 6/10
실행횟수: 7/10
실행횟수: 8/10
실행횟수: 9/10
